<a href="https://colab.research.google.com/github/minhnguyen04-HUST/Project_II_WAF_ML/blob/main/Minh_WAF_ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling and evaluation of machine learning model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install joblib

Import all dependencies


In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import pickle
import joblib
from IPython.display import display
#evaluations
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/WAF_ML_data/data.csv",index_col="index")
data

,payload,is_malicious,injection_type,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
index,,,,,,,,,,,,,
0,37662577P,0.0,LEGAL,9,0,0,50,80,56.333333,8.537499,6,0,0
1,shirting,0.0,LEGAL,8,0,0,103,116,109.000000,5.049752,7,0,1
2,&kw=%27;alert%28%27XSS%27%29;//,1.0,XSS,31,0,11,37,119,65.806452,26.617263,18,0,1
3,obeying,0.0,LEGAL,7,0,0,98,121,107.000000,7.151423,7,0,1
4,dictating,0.0,LEGAL,9,0,0,97,116,105.666667,6.599663,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110355,needlessness,0.0,LEGAL,12,0,0,100,115,107.666667,6.209312,5,1,0
110356,Mary-Jo,0.0,LEGAL,7,0,1,45,121,91.285714,25.257046,7,0,0
110357,marshaled,0.0,LEGAL,9,0,0,97,115,105.000000,6.463573,8,0,0


Selecting dependent and independent variables

In [ ]:
Y = data['is_malicious']
"""independent_variables = data.columns
independent_variables = independent_variables.delete(1)
independent_variables"""
X = data.iloc[:,3:]
X

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
results=[]

# Modeling data with Xgboost Classifier

In [ ]:
import xgboost as xgb
#?xgb.XGBClassifier()
xgb_classifer = xgb.XGBClassifier()
xgb_classifer.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
%time xgb_classifer.fit(X_train,Y_train)

CPU times: user 1.28 s, sys: 8.95 ms, total: 1.29 s
Wall time: 680 ms


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
xgb_pred = xgb_classifer.predict(X_test)

In [ ]:
results.append({
        "Model": "XGBoost",
        "Accuracy": accuracy_score(Y_test, xgb_pred),
        "Precision": precision_score(Y_test, xgb_pred, zero_division=0),
        "Recall": recall_score(Y_test, xgb_pred, zero_division=0),
        "F1 Score": f1_score(Y_test, xgb_pred, zero_division=0)
    })

In [ ]:
print(accuracy_score(Y_test, xgb_pred))
print(precision_score(Y_test, xgb_pred))
print(recall_score(Y_test, xgb_pred))
print(f1_score(Y_test, xgb_pred))

0.9994563247553462
0.9989790709545686
0.9949161159125572
0.9969434538970963


# Modeling data with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

In [ ]:
%time rf_classifier.fit(X_train, Y_train)

CPU times: user 12.8 s, sys: 9.81 ms, total: 12.8 s
Wall time: 12.9 s


RandomForestClassifier(random_state=42)

In [ ]:
rf_pred = rf_classifier.predict(X_test)
results.append({
        "Model": "Random Forest",
        "Accuracy": accuracy_score(Y_test, rf_pred),
        "Precision": precision_score(Y_test, rf_pred, zero_division=0),
        "Recall": recall_score(Y_test, rf_pred, zero_division=0),
        "F1 Score": f1_score(Y_test, rf_pred, zero_division=0)
    })


In [ ]:
print(accuracy_score(Y_test, rf_pred))
print(precision_score(Y_test, rf_pred))
print(recall_score(Y_test, rf_pred))
print(f1_score(Y_test, rf_pred))

0.9970097861544037
0.9866871479774706
0.9796644636502287
0.9831632653061224


# Modeling data with SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True, kernel='rbf', random_state=0))
])

In [ ]:
svm_pipeline.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVC(probability=True, random_state=0))])

In [ ]:
%time svm_pipeline.fit(X_train, Y_train)

CPU times: user 2min 46s, sys: 1.16 s, total: 2min 48s
Wall time: 2min 51s


Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVC(probability=True, random_state=0))])

In [ ]:
svm_pred = svm_pipeline.predict(X_test)
results.append({
        "Model": "Support Vector Machine",
        "Accuracy": accuracy_score(Y_test, svm_pred),
        "Precision": precision_score(Y_test, svm_pred, zero_division=0),
        "Recall": recall_score(Y_test, svm_pred, zero_division=0),
        "F1 Score": f1_score(Y_test, svm_pred, zero_division=0)
    })


In [ ]:
print(accuracy_score(Y_test, svm_pred))
print(precision_score(Y_test, svm_pred))
print(recall_score(Y_test, svm_pred))
print(f1_score(Y_test, svm_pred))

0.9919807901413555
0.982740021574973
0.9262836807320793
0.9536770478932217


# Modeling data with Artificial Neural Network


In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
ann_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam',
                          max_iter=300, random_state=42))
])

In [ ]:
%time ann_pipeline.fit(X_train, Y_train)

CPU times: user 55.8 s, sys: 143 ms, total: 55.9 s
Wall time: 59.1 s


Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300,
                               random_state=42))])

In [ ]:
ann_pred = ann_pipeline.predict(X_test)
results.append({
        "Model": "Neural Network",
        "Accuracy": accuracy_score(Y_test, ann_pred),
        "Precision": precision_score(Y_test, ann_pred, zero_division=0),
        "Recall": recall_score(Y_test, ann_pred, zero_division=0),
        "F1 Score": f1_score(Y_test, ann_pred, zero_division=0)
    })


In [ ]:
print(accuracy_score(Y_test, ann_pred))
print(precision_score(Y_test, ann_pred))
print(recall_score(Y_test, ann_pred))
print(f1_score(Y_test, ann_pred))

0.9954693729612178
0.9804426145136387
0.9684799186578547
0.9744245524296675


# Compare 4 models

In [ ]:
results_df = pd.DataFrame(results)
print("\nModel Comparison:\n")
print(results_df.sort_values("F1 Score", ascending=False).reset_index(drop=True))


Model Comparison:

                    Model  Accuracy  Precision    Recall  F1 Score
0                 XGBoost  0.999456   0.998979  0.994916  0.996943
1           Random Forest  0.997010   0.986687  0.979664  0.983163
2          Neural Network  0.995469   0.980443  0.968480  0.974425
3  Support Vector Machine  0.991981   0.982740  0.926284  0.953677


# Data

In [ ]:
data.head(30)

,payload,is_malicious,injection_type,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
index,,,,,,,,,,,,,
0,37662577P,0.0,LEGAL,9,0,0,50,80,56.333333,8.537499,6,0,0
1,shirting,0.0,LEGAL,8,0,0,103,116,109.000000,5.049752,7,0,1
2,&kw=%27;alert%28%27XSS%27%29;//,1.0,XSS,31,0,11,37,119,65.806452,26.617263,18,0,1
3,obeying,0.0,LEGAL,7,0,0,98,121,107.000000,7.151423,7,0,1
4,dictating,0.0,LEGAL,9,0,0,97,116,105.666667,6.599663,7,0,1
5,lafleur,0.0,LEGAL,7,0,0,97,117,106.714286,6.670067,6,0,0
6,capturers,0.0,LEGAL,9,0,0,97,117,109.444444,7.558823,8,0,0
7,8nca58z48,0.0,LEGAL,9,0,0,52,122,77.888889,26.904954,7,0,0
8,autocratic,0.0,LEGAL,10,0,0,97,117,107.100000,8.117266,7,0,0


# Storing the trained model into pickel file
to skip the training step all time and directly load trained data from file

In [ ]:
#creating binary file

with open("/content/drive/MyDrive/WAF_ML_data/mms_maf_final","wb") as file:
  pickle.dump(xgb_classifer,file)

In [ ]:
#reading binary file

with open("/content/drive/MyDrive/WAF_ML_data/mms_maf_final","rb") as file:
  xgb_classifer_pickle = pickle.load(file)

In [ ]:
#testing the model via pickle file
data['predicted_is_malicious'] = xgb_classifer.predict(X)
data['predicted_is_malicious_pickle'] = xgb_classifer_pickle.predict(X)
data[["is_malicious","predicted_is_malicious","predicted_is_malicious_pickle"]]

,is_malicious,predicted_is_malicious,predicted_is_malicious_pickle
index,,,
0,0.0,0,0
1,0.0,0,0
2,1.0,1,1
3,0.0,0,0
4,0.0,0,0
...,...,...,...
110355,0.0,0,0
110356,0.0,0,0
110357,0.0,0,0


Storing model with joblib



In [ ]:
import joblib
joblib.dump(xgb_classifer,"/content/drive/MyDrive/WAF_ML_data/mms_waf_joblib")


['/content/drive/MyDrive/WAF_ML_data/mms_waf_joblib']

In [ ]:
#prwediction via joblib
#testing the model via pickle file
xgb_classifer_joblib = joblib.load("mms_waf_joblib")
data['predicted_is_malicious_joblib'] = xgb_classifer_joblib.predict(X)
data[["is_malicious","predicted_is_malicious","predicted_is_malicious_joblib"]]

,is_malicious,predicted_is_malicious,predicted_is_malicious_joblib
index,,,
0,0.0,0,0
1,0.0,0,0
2,1.0,1,1
3,0.0,0,0
4,0.0,0,0
...,...,...,...
110355,0.0,0,0
110356,0.0,0,0
110357,0.0,0,0


# Integration with website


In [ ]:
independent_variables=['length', 'non-printable','punctuation', 'min-byte', 'max-byte', 'mean-byte', 'std-byte','distinct-byte', 'sql-keywords', 'js-keywords']
independent_variables

['length',
 'non-printable',
 'punctuation',
 'min-byte',
 'max-byte',
 'mean-byte',
 'std-byte',
 'distinct-byte',
 'sql-keywords',
 'js-keywords']

In [ ]:
independent_variables=data.columns

sql_keywords = pd.read_csv('/content/drive/MyDrive/WAF_ML_data/SQLKeywords.txt', index_col=False)
js_keywords = pd.read_csv("/content/drive/MyDrive/WAF_ML_data/JavascriptKeywords.txt",index_col=False)
def calculate_features_and_predict(payload):
  features = {}
  payload = str(payload)
  features['length'] = len(payload)
  features['non-printable'] = len([1 for letter in payload if letter not in string.printable])
  features['punctuation'] = len([1 for letter in payload if letter in string.punctuation])
  features['min-byte'] = min(bytearray(payload,'utf-8'))
  features['max-byte'] = max(bytearray(payload,'utf-8'))
  features['mean-byte'] = np.mean(bytearray(payload,'utf-8'))
  features['std-byte'] = np.std(bytearray(payload,'utf-8'))
  features['distinct-byte'] = len(set(bytearray(payload,'utf-8')))
  features['sql-keywords'] = len([1 for keyword in sql_keywords['Keyword'] if str(keyword).lower() in payload.lower()])
  features['js-keywords'] = len([1 for keyword in js_keywords['Keyword'] if str(keyword).lower() in payload.lower()])
  #payload_df = pd.DataFrame(data=features,index=[0],columns=independent_variables)
  payload_df = pd.DataFrame(features,index=[0])
  display(payload_df)
  result = xgb_classifer.predict(payload_df)
  #display(result)
  return result[0]


In [ ]:
calculate_features_and_predict("<>")

,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,2,0,2,60,62,61.0,1.0,2,0,0


np.int64(0)

In [ ]:
payload=''
while (payload != 'exit' ):
  payload = input("Enter payload")
  result = calculate_features_and_predict(payload)
  if(result > 0):
    print(f"Your payload {payload} is malicious - 403 error\n")
  else:
    print(f"Your payload {payload} is safe 200 OK\n")

Enter payload;cat /etc/passwd


,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,16,0,3,32,119,91.9375,27.806179,11,0,0


Your payload ;cat /etc/passwd is malicious - 403 error



KeyboardInterrupt: Interrupted by user